In [5]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.9 MB/s eta 0:00:0031m14.7 MB/s eta 0:00:01


In [58]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Настройка модели Hugging Face
model_name = "distilgpt2"  # Лёгкая версия GPT-2
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Перемещение модели на устройство (CPU или GPU)
device = torch.device("cpu")
model.to(device)

# Создание pipeline для генерации текста
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,  # Генерируем максимум 50 новых токенов
    temperature=0.7,    # Баланс между случайностью и детерминированностью
    top_k=50,           # Ограничение выборки токенов
    top_p=0.95,         # Фильтрация по вероятности
    device=0 if torch.cuda.is_available() else -1
)

# Инициализация LLM через HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=text_generator)

# Определение шаблонов запросов
template_1 = "Explain {topic}: {question}"
template_2 = "As a {role}, answer: {question}"

# Создание цепочек для каждого шаблона
prompt_1 = PromptTemplate(input_variables=["topic", "question"], template=template_1)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(input_variables=["role", "question"], template=template_2)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

# Работа с шаблонами
# Пример использования первого шаблона
topic = "artificial intelligence"
question = "How does a neural network work?"
response_1 = chain_1.run(topic=topic, question=question)
print("Answer (Template 1):")
print(response_1)
# Пример использования второго шаблона
role = "physicist"
question = "What is a black hole?"
response_2 = chain_2.run(role=role, question=question)
print("Answer (Template 2):")
print(response_2)

Answer (Template 1):
Explain artificial intelligence: How does a neural network work?
even light, can escape. It forms from the collapse of massive stars and is characterized by an event horizon, beyond which nothing can return.
Answer (Template 2):
As a physicist, answer: What is a black hole? A black hole is a black hole.
In the 1970s, James Dyson wrote a famous essay on the concept of black holes, which was published in The New York Times.
He was the first to write that, according


###Модель эффективно работает лишь на английском языке, на русском, она выводит несколько слов, которые могут относится к ответу и начинает их повторять